# Examples of models' creation from paper Conditional Variational AutoEncoder to Predict Suitable Conditions for Hydrogenation Reactions 

In [1]:
import numpy as np
from cvae_models import get_emb_model, get_gauss_model, generate_gauss, get_rnf_gauss_model, fit_rnf_gauss_model, get_h_model, generate_h, convert_predictions
from descriptors_generation import generate_x_descriptors, generate_y,  training_data_preparation


from CGRtools.files import RDFread

# disable tensorflow warnings
import logging, os
logging.disable(logging.WARNING)
import tensorflow as tf
tf.get_logger().setLevel('INFO')

## Data preparation

### Initial data

"data/example_data/example_hydrogenation_USPTO.rdf" - 353 previously standardized hydrogenation reactions extracted from USPTO database.

In [ ]:
with RDFread("data/example_data/example_hydrogenation_USPTO.rdf") as f:
    reaction = next(f)

In [3]:
reaction

**reaction.meta** contains data of reaction, in our case, reaction conditions:

In [4]:
for k, v in reaction.meta.items():
    print(k, v)

Condition 2_source_id US20150018393A1
Condition 2_year 2015
Condition 2_catalyst Palladium on activated carbon
Condition 3_source_id US20100197911A1
Condition 3_year 2010
Condition 3_catalyst Palladium on activated carbon
Condition 3_temperature 25.0
Condition 5_source_id US20100173954A1
Condition 5_year 2010
Condition 5_catalyst Palladium on activated carbon
Condition 6_source_id US20100267777A1
Condition 6_year 2010
Condition 6_catalyst Palladium on activated carbon
Condition 7_source_id US20130183268A1
Condition 7_year 2013
Condition 7_catalyst Palladium on activated carbon
Condition 9_source_id US20130131122A1
Condition 9_year 2013
Condition 9_catalyst Palladium on activated carbon
Condition 10_source_id US20070066542A1
Condition 10_year 2007
Condition 10_catalyst Palladium catalyst
Condition 11_source_id US20060160832A1
Condition 11_year 2006
Condition 11_catalyst Palladium on activated carbon
Condition 11_temperature 25.0
Condition 12_source_id US20060058358A1
Condition 12_year 2

Every record in RDF file must include atom-to-atom mapping and reaction conditions. RDF file shouldn`t include duplicates by transformation (reaction scheme).

It is expected that unknown or missing conditions will be skipped.

RDF file must include the field with transformation (reaction scheme) unique index, for example, the field name may be "Transformation_ID"; the field with a single catalyst name.

Every field for reaction condition should have prefix with unique condition index, with "_" as delimiter between prefix and condition name, for example, field for temperature for condition combination with index 3 will be "Condition 3_temperature".

Expected condition fields:

* **temperature** - float

* **pressure** - float

* **acids** - string with names of acids, delimiter "|"

* **bases** - string with names of bases, delimiter "|"

* **catalytic_poisons** - string with names of catalytic poisons, delimiter "|"

* **catalyst** - string with name of the single catalyst

### X descriptors generation

The following parameters for **generate_x_descriptors** must be specified: 

* **input_filename** - path to initial (but preliminarily standardized) MDL RDF file (with atom-to-atom mapping and reaction conditions)  

* **step** - the step of QSPR modeling. May be "training" or "prediction"

* **frg_filename** - if step="training", it is a path to file to save unique fragments of training set (should be used to test set fragmentation similarly preprocessing.StandardScaler() from scikit-learn). In the another case, it is the path to fitted Fragmentor file (in pickle format)

* **ipca_filename** - if step="training", it is a path to file to save PCA model. In the another case, it is the path to fitted Incremental PCA model (in pickle format). Size of chunk was defined automatically

* **x_filename** - path to file to save X descriptors (in pickle format)

* **id_field** - name of the field in RDF file with transformation (reaction scheme) unique index

* **chunksize** - number of reactions in batch for fragmentation 

* **num_pc** - number of desired principal components

Example of X generation for training set:

In [ ]:
X = generate_x_descriptors(
    input_filename="data/example_data/example_hydrogenation_USPTO.rdf",
    step="training",
    frg_filename="fragmentor.pickle", 
    ipca_filename="ipca.pickle", 
    x_filename="x_descriptors",
    id_field='Transformation_ID',
    chunksize=100,
    num_pc=100)


Fragmentation...
Fragmentation chunk No1
Fragmentation chunk No2
Fragmentation chunk No3
Fragmentation chunk No4
Fitted Fragmentor was saved to file fragmentor.pickle
Incremental PCA chunk (learning) No1 (353 reactions from 353)
Explained variance: 0.901
Fitted IncrementalPCA was saved to file ipca.pickle
Incremental PCA chunk (transformation) No1 (353 reactions from 353)


Example of X generation for test set:

In [ ]:
X = generate_x_descriptors(
    input_filename="data/example_data/example_hydrogenation_USPTO.rdf",
    step="prediction",
    frg_filename="fragmentor.pickle", 
    ipca_filename="ipca.pickle", 
    x_filename="x_descriptors.pickle",
    id_field='Transformation_ID',
    chunksize=100,
    num_pc=100)


Incremental PCA chunk (transformation) No1 (353 reactions from 353)


### Y generation

Temperature and pressure are discretized into several bins by thresholds. Thresholds should be increasing; they defined as `thresholds[i-1] <= x < thresholds[i]`

According to our paper (and this example) thresholds take the following values.


**For temperature:**

In [7]:
t_thresholds=(10, 40)

It means 't<10', '10<=t<40', 't>=40' (in Celsius degrees)

**For pressure:**

In [8]:
p_thresholds=(1, 3.5, 100)

It means 'p<1', '1<=p<3.5', '3.5<=p<100', 'p>=100' (in atmospheres).

In [ ]:

# read additives and catalysts lists from txt files
acids_bases_poisons = []
with open('data/example_data/acids_bases_poisons_list.txt', 'r') as f:
    for line in f:
        acids_bases_poisons.append(line.strip())  

catalysts = []
with open('data/example_data/catalysts_list.txt', 'r') as f:
    for line in f:
        catalysts.append(line.strip())

In [10]:
number_t_bins = len(t_thresholds) + 1
number_p_bins = len(p_thresholds) + 1
number_acids_bases_poisons = len(acids_bases_poisons)
number_catalysts = len(catalysts)

print('Number of temperature bins: ', number_t_bins)
print('Number of pressure bins: ', number_p_bins)
print('Number of additives (acids, bases, catalytic poisons): ', number_acids_bases_poisons)
print('Number of catalysts: ', number_catalysts)

Number of temperature bins:  3
Number of pressure bins:  4
Number of additives (acids, bases, catalytic poisons):  131
Number of catalysts:  227


The following parameters for **generate_y** must be specified: 

* **input_filename** - path to initial (but preliminarily standardized) MDL RDF file (with atom-to-atom mapping and reaction conditions)  

* **y_filename** - path to file to save y (in pickle format)

* **acids_bases_poisons** - list of additives (acids, bases, catalytic poisons) names (unique) 

* **catalysts** - list of catalysts names (unique) 

* **id_field** - name of the field in RDF file with transformation (reaction scheme) unique index

* **condition_id_field** - prefix of the fields in RDF file with reaction condition ("_" used as delimiter between prefix and field name). Example: "Condition 3_temperature". 


In [ ]:
y = generate_y(input_filename="data/example_data/example_hydrogenation_USPTO.rdf",
               y_filename='y.pickle', 
               acids_bases_poisons=acids_bases_poisons, 
               catalysts=catalysts, 
               id_field='Transformation_ID', 
               condition_id_field='Condition_ID',
               t_thresholds=(10, 40), 
               p_thresholds=(1, 3.5, 100), 
               none_imputer=1_000_000, 
               drop_duplicates=True)

X and y are pandas DataFrames with **id_field** as index. Numbers of principal component after PCA are columns for X. 
Temperature and pressure bins and names of additives (acids, bases, catalytic poisons) and names of catalysts are columns for y. 

In [ ]:
print(X.head())

            0         1         2         3         4         5         6   \
1569 -0.730511 -2.003991 -1.069116  0.573024  0.119204  1.717111  0.824796   
3238 -1.896448 -1.472325 -0.873939  1.487344 -0.190490  1.669372  1.085598   
3257 -1.936889 -1.476772 -0.929809  0.676508  0.178700  1.484288  0.372530   
3301 -1.065851 -1.791722 -0.609138 -0.392875 -0.789546 -0.015352  0.201867   
4747  1.081709 -1.264188 -0.978274 -1.215437 -0.475434 -0.093871  0.908365   

            7         8         9     ...           90        91        92  \
1569  0.096974  0.067021  1.319759    ...    -0.023323 -0.295685  0.084474   
3238  0.913268 -0.362688  2.163835    ...    -0.104656 -0.037896  0.015459   
3257  0.441572 -0.123229  1.261852    ...    -0.124035 -0.149829  0.122697   
3301 -1.095216 -0.271233  0.110901    ...    -0.163520  0.036292 -0.566810   
4747  0.100539 -0.002269 -0.383573    ...     0.008015  0.117902 -0.444247   

            93        94        95        96        97        

In [13]:
print(X.shape)

(353, 100)


In [14]:
print(y.head())

                  t<10 10<=t<40 t>=40 p<1 1<=p<3.5 3.5<=p<100 p>=100  \
Transformation_ID                                                      
1569                 0        0     0   0        0          0      0   
1569                 0        1     0   0        0          0      0   
1569                 0        0     0   0        0          0      0   
3238                 0        0     0   0        0          0      0   
3257                 0        0     0   0        1          0      0   

                  Ethaneperoxoic acid Acetic acid  \
Transformation_ID                                   
1569                                0           0   
1569                                0           0   
1569                                0           0   
3238                                0           0   
3257                                0           0   

                  (7,7-dimethyl-2-oxo-1-bicyclo[2.2.1]heptanyl)methanesulfonic acid  \
Transformation_ID                   

In [15]:
print(y.shape)

(394, 365)


It is necessary to join and split X and y before fitting models because for one transformation may be many conditions and convert index to MultiIndex for convenience.

In [16]:
X_train, y_train = training_data_preparation(
    X, 
    y, 
    x_final_filename="x_descriptors_final.pickle", 
    y_final_filename="y_final.pickle",
    number_t_bins=number_t_bins,
    number_p_bins=number_p_bins,
    number_acids_bases_poisons=number_acids_bases_poisons,
    number_catalysts=number_catalysts)

In [17]:
print(X_train.shape)

(394, 100)


In [18]:
print(y_train.shape)

(394, 365)


In [19]:
y_train.head()

categories    t                  p                             \
names      t<10 10<=t<40 t>=40 p<1 1<=p<3.5 3.5<=p<100 p>=100   
100341        0        0     0   0        0          0      0   
100381        0        1     0   0        0          0      0   
100381        0        0     0   0        0          0      0   
100381        0        0     0   0        0          0      0   
1004111       0        1     0   0        0          0      0   

categories acids_bases_poisons              \
names      Ethaneperoxoic acid Acetic acid   
100341                       0           0   
100381                       0           0   
100381                       0           0   
100381                       0           1   
1004111                      0           0   

categories                                                                    \
names      (7,7-dimethyl-2-oxo-1-bicyclo[2.2.1]heptanyl)methanesulfonic acid   
100341                                                      0                  
100381                                                      0                  
100381                                                      0                  
100381                                                      0                  
1004111                                                     0                  

categories              ...                                  catalysts  \
names                   ...              Ruthenium on titanium dioxide   
100341                  ...                                          0   
100381                  ...                                          0   
100381                  ...                                          0   
100381                  ...                                          0   
1004111                 ...                                          0   

categories                                                                     \
names      Dodecacarbonyltriruthenium Ruthenium (iv) oxide Ruthenium catalyst   
100341                              0                    0                  0   
100381                              0                    0                  0   
100381                              0                    0                  0   
100381                              0                    0                  0   
1004111                             0                    0                  0   

categories                                                          \
names      Urushibara nickel Palladium acetate on alumina calcined   
100341                     0                                     0   
100381                     0                                     0   
100381                     0                                     0   
100381                     0                                     0   
1004111                    0                                     0   

categories                                                     \
names      Chiral iridium complex of type PIr(N)(COD) + BArF-   
100341                                                      0   
100381                                                      0   
100381                                                      0   
100381                                                      0   
1004111                                                     0   

categories                                                        \
names      Palladium dihydroxide Palladium on aluminum hydroxide   
100341                         0                               0   
100381                         0                               0   
100381                         0                               0   
100381                         0                               0   
1004111                        0                               0   

categories                                
names      Palladium on barium hydroxide  
100341                                 0  
100381              

## Embedded model

An embedded model - a pre-trained model with an architecture like the LRM model from our previous paper [[link]](https://doi.org/10.3390/ijms23010248) - will be used as a part of our other models - g-CVAE, rnf-CVAE and h-CVAE.

In [20]:
embedded_model = get_emb_model(in_dim=100, out_dim=365)

In [21]:
embedded_model.fit(X_train, y_train, batch_size=10, epochs=1)

Train on 394 samples
394/394 [==============================] - 0s 824us/sample - loss: 0.0656 - acc: 0.9766


In [22]:
embedded_model.save('embedded_model.h5')

## g-CVAE model (with Gauss random distribution)

### Build and fit model

In [23]:
gauss_model, decoder = get_gauss_model(
    trained_emb_model_path='embedded_model.h5', 
    in_dim=100, 
    out_dim=365, 
    temperature_dim=3,
    pressure_dim=4,
    adds_dim=131,
    catalyst_dim=227)

In [24]:
gauss_model.fit([X_train, y_train], 
                [y_train['t'], y_train['p'], 
                y_train['acids_bases_poisons'], y_train['catalysts']], 
                batch_size=10, epochs=1)

Train on 394 samples
394/394 [==============================] - 1s 3ms/sample - loss: 20.7098 - model_3_loss: 0.5125 - model_3_1_loss: 0.5512 - model_3_2_loss: 18.7480 - model_3_3_loss: 0.6325


### Prediction

In [25]:
top_ans = generate_gauss(X, decoder, adds_dim=131, catalyst_dim=227, pressure_dim=4, temperature_dim=3, s=100, top_k=10)

100/100

In [26]:
top_ans.shape

(353, 10, 134)

In [27]:
predictions = convert_predictions(top_ans, X, y_train.columns)

In [28]:
predictions.head()

ranks,1,2,3,4,5,6,7,8,9,10
Transformation_ID,,,,,,,,,,
1569,"10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon"
3238,"10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon"
3257,"10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon"
3301,"10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon"
4747,"t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon"


In [29]:
predictions.loc['1569']

ranks
1     10<=t<40, p>=100, Palladium on activated carbon
2     10<=t<40, p>=100, Palladium on activated carbon
3     10<=t<40, p>=100, Palladium on activated carbon
4     10<=t<40, p>=100, Palladium on activated carbon
5     10<=t<40, p>=100, Palladium on activated carbon
6     10<=t<40, p>=100, Palladium on activated carbon
7     10<=t<40, p>=100, Palladium on activated carbon
8     10<=t<40, p>=100, Palladium on activated carbon
9     10<=t<40, p>=100, Palladium on activated carbon
10    10<=t<40, p>=100, Palladium on activated carbon
Name: 1569, dtype: object

In [30]:
predictions.loc['1569', 1]

'10<=t<40, p>=100, Palladium on activated carbon'

## rnf-CVAE model (with Riemannian Normalizing Flow distribution)

### Build and fit model

In [34]:
rnf_model, decoder = get_rnf_gauss_model(
    trained_emb_model_path='embedded_model.h5', in_dim=100, out_dim=365, adds_dim=131, catalyst_dim=227, pressure_dim=4, temperature_dim=3)

In [38]:
fit_rnf_gauss_model(rnf_model, epochs=1, batch_size=10, X_train=X_train.values, y_train=y.values, t_dim=3, p_dim=4, add_dim=131)

-------------------------------------------------------------
1
-------------------------------------------------------------
Train on 394 samples
 10/394 [..............................] - ETA: 1s - loss: 5.9581 - model_11_loss: 0.4296 - model_11_1_loss: 0.5224 - model_11_2_loss: 0.3338 - model_11_3_loss: 0.2078

394/394 [==============================] - 1s 3ms/sample - loss: 6.5424 - model_11_loss: 0.4948 - model_11_1_loss: 0.5332 - model_11_2_loss: 0.7417 - model_11_3_loss: 0.2640


### Prediction

In [40]:
top_ans = generate_gauss(X, decoder, adds_dim=131, catalyst_dim=227, pressure_dim=4, temperature_dim=3, s=100, top_k=10)

100/100

In [41]:
predictions = convert_predictions(
    top_ans=top_ans,
    X_test=X, 
    y_train_final_columns=y_train.columns,
    id_field="Transformation_ID"
    )

In [43]:
predictions.tail()

ranks,1,2,3,4,5,6,7,8,9,10
Transformation_ID,,,,,,,,,,
1333123,"t>=40, 3.5<=p<100, Palladium on activated carbon","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c..."
1335141,"t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon"
1336812,"10<=t<40, 3.5<=p<100, Palladium on activated c...","t>=40, 3.5<=p<100, Palladium on activated carbon","t>=40, 3.5<=p<100, Palladium on activated carbon","t>=40, 3.5<=p<100, Palladium on activated carbon","t>=40, 3.5<=p<100, Palladium on activated carbon","t>=40, 3.5<=p<100, Palladium on activated carbon","t>=40, 3.5<=p<100, Palladium on activated carbon","t>=40, 3.5<=p<100, Palladium on activated carbon","t>=40, 3.5<=p<100, Palladium on activated carbon","t>=40, 3.5<=p<100, Palladium on activated carbon"
1339806,"10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon"
1339852,"10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c..."


In [44]:
predictions.loc['1339852']

ranks
1       10<=t<40, p>=100, Palladium on activated carbon
2     10<=t<40, 3.5<=p<100, Palladium on activated c...
3     10<=t<40, 3.5<=p<100, Palladium on activated c...
4     10<=t<40, 3.5<=p<100, Palladium on activated c...
5     10<=t<40, 3.5<=p<100, Palladium on activated c...
6     10<=t<40, 3.5<=p<100, Palladium on activated c...
7     10<=t<40, 3.5<=p<100, Palladium on activated c...
8     10<=t<40, 3.5<=p<100, Palladium on activated c...
9     10<=t<40, 3.5<=p<100, Palladium on activated c...
10    10<=t<40, 3.5<=p<100, Palladium on activated c...
Name: 1339852, dtype: object

## h-CVAE model (with Hyperspherical Uniform distribution)

### Build and fit model

In [45]:
h_model, decoder = get_h_model(
    trained_emb_model_path='embedded_model.h5', 
    in_dim=100, 
    out_dim=365, adds_dim=131, catalyst_dim=227, pressure_dim=4, temperature_dim=3)

In [46]:
h_model.fit([X_train, y_train], 
                [y_train['t'], y_train['p'], 
                y_train['acids_bases_poisons'], y_train['catalysts']], 
                batch_size=10, epochs=1)

Train on 394 samples
394/394 [==============================] - 3s 6ms/sample - loss: 20.3497 - model_16_loss: 0.5130 - model_16_1_loss: 0.5444 - model_16_2_loss: 18.4815 - model_16_3_loss: 0.5345


### Prediction

In [47]:
top_ans = generate_h(X, decoder, adds_dim=131, catalyst_dim=227, pressure_dim=4, temperature_dim=3, s=100, top_k=10)

100/100

In [48]:
predictions = convert_predictions(
    top_ans=top_ans,
    X_test=X,
    y_train_final_columns=y_train.columns,
    id_field="Transformation_ID"
)

In [49]:
predictions.head()

ranks,1,2,3,4,5,6,7,8,9,10
Transformation_ID,,,,,,,,,,
1569,"t>=40, 3.5<=p<100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon"
3238,"t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon","t>=40, p>=100, Palladium on activated carbon"
3257,"10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon"
3301,"10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon","10<=t<40, p>=100, Palladium on activated carbon"
4747,"t>=40, 3.5<=p<100, Palladium on activated carbon","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c...","10<=t<40, 3.5<=p<100, Palladium on activated c..."


In [50]:
predictions.loc['1569']

ranks
1      t>=40, 3.5<=p<100, Palladium on activated carbon
2          t>=40, p>=100, Palladium on activated carbon
3     10<=t<40, 3.5<=p<100, Palladium on activated c...
4       10<=t<40, p>=100, Palladium on activated carbon
5       10<=t<40, p>=100, Palladium on activated carbon
6       10<=t<40, p>=100, Palladium on activated carbon
7       10<=t<40, p>=100, Palladium on activated carbon
8       10<=t<40, p>=100, Palladium on activated carbon
9       10<=t<40, p>=100, Palladium on activated carbon
10      10<=t<40, p>=100, Palladium on activated carbon
Name: 1569, dtype: object